In [12]:
from pyspark.sql import SparkSession
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType, FloatType, StructType, StructField
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import spacy
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
from textblob import TextBlob
from sentence_transformers import SentenceTransformer
import subprocess
import os

In [13]:
# Disable SSL verification
os.environ['CURL_CA_BUNDLE'] = ""

In [ ]:
#pip install pandas numpy pyspark nltk spacy transformers textblob sentence-transformers

In [3]:
# Download NLTK data
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\uazhyhalbo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\uazhyhalbo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
# Initialize Spark session
spark = SparkSession.builder.appName("0000.parquet").getOrCreate()

In [6]:
# Read Parquet file into DataFrame
df = spark.read.parquet("0000.parquet", header=True, inferSchema=True)

In [7]:
# Show DataFrame
df.show()

+--------------------+-----+
|                text|label|
+--------------------+-----+
|I rented I AM CUR...|    0|
|"I Am Curious: Ye...|    0|
|If only to avoid ...|    0|
|This film was pro...|    0|
|Oh, brother...aft...|    0|
|I would put this ...|    0|
|Whoever wrote the...|    0|
|When I first saw ...|    0|
|Who are these "Th...|    0|
|This is said to b...|    0|
|It was great to s...|    0|
|I can't believe t...|    0|
|Never cast models...|    0|
|Its not the cast....|    0|
|Today I found "Th...|    0|
|This film is just...|    0|
|My interest in Do...|    0|
|I have this film ...|    0|
|I think I will ma...|    0|
|Pros: Nothing<br ...|    0|
+--------------------+-----+
only showing top 20 rows



In [17]:
# Load spaCy model
nlp = spacy.load("en_core_web_sm")

OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a Python package or a valid path to a data directory.

In [32]:
#Sentiment Analysis
# Use a pipeline as a high-level helper
sentiment_analyzer = pipeline("text-classification", model="SamLowe/roberta-base-go_emotions")

def analyze_sentiment(text):
    result = sentiment_analyzer(text)[0]
    return result['label'], float(result['score'])

analyze_sentiment_udf = udf(analyze_sentiment, StructType([
    StructField("sentiment", StringType(), True),
    StructField("sentiment_score", FloatType(), True)
]))

df = df.withColumn("sentiment_analysis", analyze_sentiment_udf(df['text']))
df = df.withColumn("sentiment", df['sentiment_analysis'].getItem("sentiment"))
df = df.withColumn("sentiment_score", df['sentiment_analysis'].getItem("sentiment_score"))
df = df.drop("sentiment_analysis")


config.json:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

c:\Users\uazhyhalbo\AppData\Local\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\uazhyhalbo\.cache\huggingface\hub\models--SamLowe--roberta-base-go_emotions. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/380 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [34]:
nlp = spacy.load("en_core_web_sm")

def extract_entities(text):
    doc = nlp(text)
    return [(ent.text, ent.label_) for ent in doc.ents]

extract_entities_udf = udf(lambda text: extract_entities(text), ArrayType(StructType([
    StructField("entity", StringType(), True),
    StructField("label", StringType(), True)
])))

df = df.withColumn("entities", extract_entities_udf(df['text']))


OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a Python package or a valid path to a data directory.

In [39]:
pip install spacy

Note: you may need to restart the kernel to use updated packages.
